In [73]:
#add necessary libraries
import networkx as nx #library supporting networks
import matplotlib.pyplot as plt #plotting
import pandas as pd
import numpy as np
import scipy.stats as stat
from scipy import optimize
#make sure plots are embedded into the notebook
%pylab inline 
import statsmodels.formula.api as smf
#import shapefile as shp
#from shapely.geometry.polygon import Polygon
from descartes import PolygonPatch
import os
from networkx.algorithms import community
from sklearn.mixture import GaussianMixture 

# Import required libraries
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Dropout
from keras import regularizers
from keras import optimizers
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import set_random_seed
from keras import backend as K
import datetime
from sklearn.mixture import GaussianMixture
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import operator

%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


/Users/hemingyi/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['partition', 'matrix', 'f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [54]:
RecordWritingPath = '/Users/hemingyi/Documents/capstone/'
TransportationDataPath = '/Users/hemingyi/Documents/capstone/transportation/'
EventDataPath = '/Users/hemingyi/Documents/capstone/event data/new/'
comboPath = '/Users/hemingyi/Documents/capstone/combo/'
# dataFile = TransportationDataPath+city+'EdgeYearwiseAggregated.csv'


In [55]:
def readTransDF(TransportationDataPath,city):
    df = pd.read_csv(TransportationDataPath+city+'EdgeDatewiseAggregated.csv')
    return df
def makeGraphfromDf(df):
    G=nx.DiGraph()
    nx.set_edge_attributes(G,'weight', 0)
    for k in df.index:
        G.add_edge(df['start_id'][k],df['end_id'][k],weight=df['amount'][k])
#     nx.write_edgelist(G, comboPath+'temp/%s.net'%city)
    return G

In [56]:
def getComboPartition(G,comboPath,city):
    #save network in net format
    nodenum={}
#     G = makeGraphfromDf(df)
    #create a dictionary transforming nodes to unique numbers
    nodes = list(G.nodes())
    print('nodes amount: ',len(nodes))
    for i,j in enumerate(list(G.nodes())):
        nodenum[str(j)] = str(i)
#         nodes[str(i)] = str(j)
#     i=0
#     for n in list(G.nodes()):
#         nodenum[n]=i
#         nodes[i]=n
#         i+=1

    tempNetFile = comboPath+'temp/%s.net'%city
    f = open(tempNetFile, 'w')
    f.write('*Arcs\n')

    for e in G.edges(data=True):
        f.write('{0} {1} {2}\n'.format(nodenum[str(e[0])],nodenum[str(e[1])],e[2]['weight']))
    f.close()

    #run combo
    command= comboPath+'/comboCPP '+tempNetFile#+' '+str(maxcom)
    os.system(command)

    #read resulting partition
    partitionFile = comboPath+'temp/'+city + '_comm_comboC++.txt'
    f = open(partitionFile, 'r')
    i=0
    partition={}
    for line in f:
        partition[str(nodes[i])]=str(int(line))
        i+=1
#         print(i)
    f.close()
    os.remove(partitionFile) 
    os.remove(tempNetFile)
#     print("Communities: ",len(set(partition.values())))
    return partition 

In [57]:
def getSubCommunity(partition,df):
    df['start_community'] = df['start_id'].apply(lambda x: partition[str(x)])
    df['end_community'] = df['end_id'].apply(lambda x: partition[str(x)])
    data2 = df[df.start_community == df.end_community]
    communities = list(set(df.start_community))
    print('communities numbers: ',communities)
    SubPartition = {}
    for c in communities:
        print('detecting sub-communities in community ',c)
        d = data2[data2.start_community == c]
        graph = makeGraphfromDf(d)
        p = getComboPartition(graph,comboPath,city)
        SubPartition[c] = p
    return (SubPartition,df)

In [58]:
def aggregateByCommunities(partition,SubPartition,city,TransportationDataPath):
    data = pd.read_csv(TransportationDataPath+city+'EdgeDatewiseAggregated.csv')

    print('Raw shape: ',data.shape)
    if 'Date' in data.columns:
        data['date'] = pd.to_datetime(data.Date)
    elif 'date' in data.columns:
        data['date'] = pd.to_datetime(data.date)
    else:
        print(data.columns)

    data['start_community'] = data['start_id'].apply(lambda x: partition[str(x)])
    data['end_community'] = data['end_id'].apply(lambda x: partition[str(x)])
    data['subStart'] = data.apply(lambda x: str(x['start_community'])+'.'+str(SubPartition[x['start_community']][str(x['start_id'])]),axis=1)
    data['subEnd'] = data.apply(lambda x: str(x['end_community'])+'.'+str(SubPartition[x['end_community']][str(x['end_id'])]),axis=1)

    communityData = data[['subStart','subEnd','date','amount']]
    communityData = communityData.groupby(['subStart','subEnd','date']).sum().reset_index()
    communityData.columns = ['start_id', 'end_id', 'date', 'amount']

    return communityData

In [59]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='amount', index=['date'],
                    columns=['start_id','end_id'], aggfunc=np.sum, fill_value=0)
    return table

In [60]:
def getAutoEncoder(input_dim,encoding_dim):
    #define autoencoder architecture
    autoencoder = Sequential()
    autoencoder.add(Dense(encoding_dim[0], input_shape=(input_dim,), activation='relu'))

    for l in range(1,len(encoding_dim)):
        #autoencoder.add(Dropout(0.1))
        autoencoder.add(Dense(encoding_dim[l], input_shape=(encoding_dim[l-1],), activation='relu'))
    
    for l in range(len(encoding_dim)-1,1,-1):
        autoencoder.add(Dense(encoding_dim[l-1], input_shape=(encoding_dim[l],), activation='relu'))

    autoencoder.add(Dense(input_dim, input_shape=(encoding_dim[0],), activation='sigmoid'))

    autoencoder.summary()
    return autoencoder

In [61]:
#extract encoder layers from the networks
def encoderLayers(autoencoder,input_,n_layer):
    output_=input_
    for l in range(n_layer):
        output_=autoencoder.layers[l](output_)
    return output_

In [62]:
# import events data
def getEvents(EventDataPath,city):
    events_data =EventDataPath+city+'Events.csv'
    df_events = pd.read_csv(events_data, encoding = "ISO-8859-1", parse_dates=['Date'], infer_datetime_format=True)

    # dataframe for events
    df_finalEvents =  df_events[['Date', 'Type']]

    # list events
    lis_event = df_finalEvents['Type'].unique()
    lis_event = list(lis_event)
    return (lis_event,df_finalEvents)

In [63]:
def AutoEncoderMatrix(matrix):
    # Set random seeds
    np.random.seed(112)
    set_random_seed(22)
    input_dim = matrix.shape[1]
    encoding_dim = [500,100,15]
    autoencoder = getAutoEncoder(input_dim,encoding_dim)
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    history = autoencoder.fit(matrix, matrix,
                    epochs=80,
                    batch_size=50,
                    shuffle=True,
                    verbose=1)
    input_layer = Input(shape=(input_dim,))
    encoder = Model(input_layer, (encoderLayers(autoencoder,input_layer,len(encoding_dim))))
    reducedMatrix = encoder.predict(matrix)
    return reducedMatrix

In [64]:
def anomalyDetection(y,pval = 0.2,iterN=20):
    #index of regular (non-outlier points)
    #rind=y[:,0]>-10 
    rind = np.array(range(y.shape[0]))
    
    #clustering model
    gm=GaussianMixture(n_components=3,n_init=100,max_iter=1000,random_state=0) 
    for i in range(iterN): #iterate
        print('Iteration {}'.format(i+1))  
        clustering=gm.fit(y[rind,:]) #fit EM clustering model excluding outliers
        l=clustering.score_samples(y) #estimate likelihood for each point
        Lthres=sorted(l)[int(len(l)*pval)] #anomaly threshold
        rind0=0+rind
        rind=l>Lthres #non-anomalous points
        if all(rind==rind0):
            print('Convergence in {} iterations'.format(i+1))
            break
    return l < Lthres

In [72]:
f = open(RecordWritingPath+'F1ScoreAutoEncoderCommunity.txt', 'w+')
f.write('Auto Encoder\n')
for city in ['DC','Taipei','Chicago','NewYork']:
    f = open(RecordWritingPath+'F1ScoreAutoEncoderCommunity.txt', 'a+')
    print(city)
    f.write(city+'\n')
    
    df = readTransDF(TransportationDataPath,city)
    lis_event,df_finalEvents = getEvents(EventDataPath,city)
    G = makeGraphfromDf(df)
    partition = getComboPartition(G,comboPath,city)
    SubPartition,df = getSubCommunity(partition,df)
    data = aggregateByCommunities(partition,SubPartition,city,TransportationDataPath)
    dataTs = getTimeSeries(data)
    matrix = dataTs.values
    matrix = np.log(matrix+1)
    for i in range(matrix.shape[1]):
        matrix[:, i] = (matrix[:, i] - matrix[:, i].min()) / (matrix[:, i].max() - matrix[:, i].min())
    matrix = AutoEncoderMatrix(matrix)
    date = dataTs.index.to_frame().rename(columns={'date':'Date'})
    result = {}
    for thres in range(1,10, 1):
        th = thres/10
        print("Threshhold: ",th)
        outliers = anomalyDetection(matrix,pval = th)
        EventsDF = df_finalEvents['Date'].drop_duplicates().to_frame()
        EventsDF['Anomaly'] = True
        date['Date'] = date['Date'].astype('str')
        EventsDF['Date'] = EventsDF['Date'].astype('str')
        df = EventsDF.merge(date,on='Date',how='right')
        df['outlier'] = outliers
        Precision = len(df[(df['outlier']==True)&(df['Anomaly']==True)])/len(df[df['outlier']==True])
        Recall = len(df[(df['outlier']==True)&(df['Anomaly']==True)])/len(EventsDF)
        if Precision+Recall > 0:
            F1 = (2*Precision*Recall)/(Precision+Recall)
        else:
            F1 = 0
        result[th] = F1
    bestTh = max(result.items(), key=operator.itemgetter(1))[0]
    f.write('overall F1: '+str(result[bestTh])+', Threshold: '+str(bestTh)+'\n')
    f.write('event,single_type_recall,single_type_FPR\n')
    EventType = list(df_finalEvents['Type'].unique())
    outliers = anomalyDetection(matrix,pval = bestTh)
    for event in EventType:
        print(event)
        SingleEventDF = df_finalEvents[df_finalEvents['Type'] == event]
        SingleEventDF = SingleEventDF.drop_duplicates()
        date['Date'] = date['Date'].astype('str')
        SingleEventDF['Date'] = SingleEventDF['Date'].astype('str')
        SingleDF = date.merge(SingleEventDF, on='Date', how='left')
    #     SingleDF = SingleEventDF.merge(date,on='Date',how='left')
        SingleDF['outliers'] = outliers
    #     SinglePrecision = len(SingleDF[(SingleDF['outlier']==True)&(SingleDF['Type'].notnull)])/len(SingleDF[SingleDF['outlier']==True])
        SingleRecall = len(SingleDF[(SingleDF['outliers']==True)&(SingleDF['Type'].notnull())])/len(SingleEventDF)
        SingleFPR = len(SingleDF[(SingleDF['outliers']==True)&(SingleDF['Type'].isna())])/len(SingleDF[SingleDF['Type'].isna()])
        print(SingleRecall, SingleFPR)
        f.write(event+','+str(SingleRecall)+','+str(SingleFPR)+'\n')
    f.close()

DC
nodes amount:  46
communities numbers:  ['1', '2', '0']
detecting sub-communities in community  1
nodes amount:  6
detecting sub-communities in community  2
nodes amount:  34
detecting sub-communities in community  0
nodes amount:  6
Raw shape:  (639177, 4)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 500)               61000     
_________________________________________________________________
dense_42 (Dense)             (None, 100)               50100     
_________________________________________________________________
dense_43 (Dense)             (None, 15)                1515      
_________________________________________________________________
dense_44 (Dense)             (None, 100)               1600      
_________________________________________________________________
dense_45 (Dense)             (None, 121)               12221     
Total params:

Iteration 4
Iteration 5
Iteration 6
Iteration 7
Convergence in 7 iterations
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Convergence in 9 iterations
Threshhold:  0.5
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Convergence in 6 iterations
Threshhold:  0.6
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Convergence in 11 iterations
Threshhold:  0.7
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Convergence in 7 iterations
Threshhold:  0.8
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Convergence in 9 iterations
Threshhold:  0.9
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Convergence in 6 iterations
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Ite

638/638 [==============================] - 0s 202us/step - loss: 0.5043
Epoch 69/80
638/638 [==============================] - 0s 238us/step - loss: 0.5046
Epoch 70/80
638/638 [==============================] - 0s 235us/step - loss: 0.5060
Epoch 71/80
638/638 [==============================] - 0s 314us/step - loss: 0.5052
Epoch 72/80
638/638 [==============================] - 0s 247us/step - loss: 0.5045
Epoch 73/80
638/638 [==============================] - 0s 245us/step - loss: 0.5043
Epoch 74/80
638/638 [==============================] - 0s 429us/step - loss: 0.5043
Epoch 75/80
638/638 [==============================] - 0s 230us/step - loss: 0.5045
Epoch 76/80
638/638 [==============================] - 0s 186us/step - loss: 0.5041
Epoch 77/80
638/638 [==============================] - 0s 206us/step - loss: 0.5061
Epoch 78/80
638/638 [==============================] - 0s 216us/step - loss: 0.5048
Epoch 79/80
638/638 [==============================] - 0s 195us/step - loss: 0.5049
Epoc

730/730 [==============================] - 0s 369us/step - loss: 0.4769
Epoch 41/80
730/730 [==============================] - 0s 368us/step - loss: 0.4767
Epoch 42/80
730/730 [==============================] - 0s 368us/step - loss: 0.4767
Epoch 43/80
730/730 [==============================] - 0s 503us/step - loss: 0.4759
Epoch 44/80
730/730 [==============================] - 0s 364us/step - loss: 0.4770
Epoch 45/80
730/730 [==============================] - 0s 370us/step - loss: 0.4771
Epoch 46/80
730/730 [==============================] - 0s 373us/step - loss: 0.4758
Epoch 47/80
730/730 [==============================] - 0s 367us/step - loss: 0.4767
Epoch 48/80
730/730 [==============================] - 0s 377us/step - loss: 0.4759
Epoch 49/80
730/730 [==============================] - 0s 387us/step - loss: 0.4764
Epoch 50/80
730/730 [==============================] - 0s 492us/step - loss: 0.4762
Epoch 51/80
730/730 [==============================] - 0s 366us/step - loss: 0.4765
Epoc

In [ ]:
#Synthetic Data